# Análise Comparativa de Modelos

Nesse notebook iremos olhar para os textos obtidos na primeira unidade para construir modelos de aprendizado de máquina. Para isso, precisaremos de algumas bibliotecas já conhecidas (*pandas, nltk, numpy*) e de outras novas, relacionadas à:
- preparação dos dados: 
    - *CountVectorizer*, 
    - *TfidfVectorizer*,
    - *TruncatedSVD*
    - *StandardScaler*
- aprendizagem em si: 
    - *LogisticRegression*, 
    - *KNeighborsClassifier*, 
    - *SVC*, 
    - *RandomForestClassifier*
- construção da sequência de passos a realizar:
    - *Pipeline*
- avaliação e seleção dos modelos obtidos:
    - *ShuffleSplit*,
    - *cross_validate*
    
Algumas dessas funções serão explicadas adiante. Sigamos.

## Importando bibliotecas

In [1]:
from itertools import product # produto cartesiano de duas listas sem precisar de for aninhados
import pandas as pd # manipulação de dataframes
import nltk # ferramentas p/ processamento de linguagem natural
import numpy as np # manipulação de matrizes, funções matemáticas
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    ShuffleSplit, KFold, cross_validate, RandomizedSearchCV
)

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/samya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Carregando Dados

In [2]:
# Adquire dados do arquivo produzido na etapa anterior
input_path = "../data/interim/news.csv"
df = pd.read_csv(input_path)

# Converte as colunas com texto em uma lista
corpus = df.text.to_list()

# Atribui os valores 1 para notícias verdadeiras e 0 para notícias falsas
labels = df.label.replace({"true": 1, "fake": 0})

## Conversão dos textos (Vetorizadores)

### CountVectorizer

Esse método de conversão cria uma matriz com a contagem de todas as palavras presentes em todos os textos considerados. Esse método também é chamado de Bag of Words, ou Saco de Palavras.

![CountVectorizer](images/count-vectorizer.png)

### TfidfVectorizer

Esse método de conversão cria uma matriz que considera dois valores: TF e DF. TF ($tf_{i,j}$) é a contagem de uma palavra ($i$) no texto ($j$), tal como é calculada pelo CountVectorizer. Já DF ($df_{i}$) diz respeito à quantidade de textos em que aquela palavra aparece. No cálculo, o valor para uma dada palavra ($w_{i,j}$) é dado por:

$ w_{i,j} = tf_{i,j} \times log(\frac{N}{df_{i}}) $

Em que $N$ é o número total de documentos. Como $df_{i}$ está no denominador do logarítmo, trata-se de uma conversão que considera TF e o inverso de DF, iDF.

![CountVectorizer](images/tfidf-vectorizer.png)

Em ambos os casos, palavras comuns e sem importância semântica no idioma trabalhado devem ser removidas. Afinal, caso fossem consideradas na conversão TFiDF, as palavras "de" e "desse" no primeiro texto teriam o mesmo peso da palavra "corrupção" e "suspeito", com importância semântica superior.

Aqui, optamos também por utilizar apenas as 1000 palavras mais frequentes em ambas as conversões, de modo a limitar o número de características para o treinamento.

In [3]:
stop_words = nltk.corpus.stopwords.words('portuguese')
vectorizer = Pipeline([
    ('count', CountVectorizer()),
    ('tfidf', TfidfTransformer())
])

## Modelos de Aprendizagem de Máquina

### Regressão Logística

A regressão logística, que é um modelo classificador, utiliza uma função logística para calcular as probabilidades de uma notícia ser falsa ou verdadeira. A função logística por trás deste modelo é uma curva sigmoide contendo as probabilidades de classificação em uma determinada categoria (notícia falsa ou verdadeira). A partir disso se encontra um valor limite dentro da curva para que então se obtenham as predições.

### KNN

Aqui a classificação das notícias em falsas ou verdadeira é feita com base nos K-vizinhos mais próximos (*K-Nearest Neighbors*) a ela. Se utilizamos $K = 10$ e dentre as 10 notícias mais próximas de uma notícia $i$ no nosso espaço dimensional, 6 são falsas e 4 são verdadeiras, esta notícia $i$ será classificada como falsa. No nosso caso utilizamos $K = 21$.

### SVC

O SVC é um *Support Vector Classifier*, isso é, um algoritmo de classificação que utiliza vetores (dados) como suporte para definir um hiperplano que separe dados de um grupo dos dados de outro grupo. No nosso caso, o SVC busca um hiperplano que separe as matrizes produzidas a partir de notícias falsas daquelas de notícias verdadeiras.

O hiperplano pode ser obtido após uma transformação dos dados, na qual eles saem de seu espaço dimensional original, intrínseco, e são levados a um espaço dimensional superior. Essa transformação é feita através através de diversas fórmulas matemáticas para que então o hiperplano ideal - aquele que separa os dados com melhor margem - seja identificado; este é o chamado *kernel trick*.

### Random Forest

A *Random Forest*, uma floresta aleatória, é um método que faz uso de diversas Árvores de Decisão, *Decision Trees*. As árvores de decisão são aquelas que fazem sucessivas divisões nos dados utilizando as características disponíveis de modo a obter grupos que sejam o mais diferentes entre si e o mais coesos internamente. Exemplo de divisão para separação de grupos: textos que têm a palavra ministro e textos que não têm, textos contendo a palavra corrupção e textos que não tem.

Contando então com várias árvores de decisão, a *Floresta*, é um classificador do tipo *ensemble*, pois considera diferentes predições finais para cada texto pelas diferentes árvores. A predição prevalente para um dado texto é então incorporada à saída da *Random Forest*. Como as árvores de decisão não são dependentes entre si, possuindo critérios próprios e não relacionados, a Random Forest minimiza o viés que viria do uso de uma única árvore, se tornando um modelo poderoso.


In [5]:
models = [{
    'name': 'LR',
    'model': LogisticRegression(),
    'parameters':{
        'penalty': ['l1', 'l2'],
        'C' : np.logspace(-4, 4, 10),
    }
},{
    'name': 'KNN',
    'model': KNeighborsClassifier(),
    'parameters': {
        'n_neighbors': np.arange(3, 21, 2),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
},{
    'name': 'SVC',
    'model': SVC(max_iter=10000, gamma='auto'),
    'parameters':{
        "C": [1, 10, 100, 1e3, 1e4, 1e5]
    }
},{
    'name': 'RF',
    'model': RandomForestClassifier(),
    'parameters':{
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy'],
        'min_samples_leaf': [1, 2, 4],
        'min_samples_split': [2, 5, 10],
        'bootstrap': [True, False]
    }    
}]

## Redução de Dimensionalidade

Quando trabalhamos com texto é natural que o resultado das transformações dos textos em vetores (CountVectorizer, TFiDFVectorizer, etc) seja uma matriz esparsa, pois grande parte das palavras NÃO estará presente em diversos textos. Desse modo, pode ser vantajoso aplicar uma redução de dimensionalidade. Na etapa de transformação, ficamos com mil *features*, referentes às ocorrências das palavras mais frequentes. 

Agora, usaremos a função TruncatedSVD para reduzir nossa matriz esparsa pela metade, ficando apenas com as ocorrências das 500 palavras menos esparsas dentre as 1000 mais frequentes. A decomposição em valores singulares (*singular value decomposition*, SVD) transforma a matriz recebida e a leva para um espaço dimensional menor, reduzindo o efeito de sinônimos e palavras similares, as quais são usadas de maneira alternada pelos textos.

## Normalização

Na normalização os valores são transformados de forma a escala das diferentes características, isso reduz as chances de superestimação ou subestimação de determinadas *features* em função de seus valores padrão. Aqui, foi empregada uma normalização do tipo Z ou padronização, que subtrai a média da distribuição do para cada valor e divide o resultado pelo desvio padrão para aquela característica. Essa é uma maneira de normalizar sem alterar a distribuição das features, seja qual for.

In [6]:
pca = TruncatedSVD()
scaler = StandardScaler()

## Pipeline

Para facilitar a comparação entre os modelos utilizamos a função *Pipeline*. O protocolo envolve produzir predições com todas as combinações de transformadores e todos os modelos de aprendizagem de máquina produzidos com os mesmos métodos de redução de dimensionalidade e normalização.

## Validação Cruzada

Para testar a robustez dos modelos, a validação cruzada foi feita sobre 10 divisões aleatórias dos dados, conservando 80% deles para o conjunto de treinamento e 20% para teste. Ao final, os escores de avaliação (acurácia, F1-score, **baba** e **bebe**) obtidos foram armazenados em uma variável *results* para posterior análise.

### Acurácia

Esta medida de avaliação considera os acertos que o modelo realizou, sejam eles ao classificar notícias falsas como falsas ou notícias verdadeiras como verdadeiras. Todos os acertos são computados e calculados em forma de porcentagem em relação ao número total de dados. Esta é uma métrica interessante quando ambas as categorias são igualmente importantes.

### F1

No nosso caso, como detector de notícias falsas, pode ser mais interessante, caso seja necessário, priorizar modelos que tenham melhor desempenho em corretamente rotular notícias falsas. Nesse caso, o *F1-score* é uma boa medida para a avaliação dos erros (falsos positivos e falsos negativos) e será dado pela média harmônica entre precisão e sensibilidade. No nosso caso:

- Precisão
> Proporção dada pelo # de notícias falsas corretamente classificadas como falsas e o # total de notícias classificadas como falsas.
- Sensibilidade
> Proporção dada pelo # de notícias corretamente classificadas como falsas dividido pelo # notícias realmente falsas. 

In [7]:
results = {}
n_splits_cv = 10
n_splits_gs = 5

split_cv = ShuffleSplit(n_splits=n_splits_cv, test_size=.2)
for model_name, model_ in models.items():
    print(model_name)
    model_params = {
        f'model__{key}': value for key, value in params[model_name].items()
    }
    param_distributions = {
        'vectorizer__tfidf__use_idf': [False, True],
        'vectorizer__count__max_features': [1000, 2000],
        'pca__n_components': [100, 200, 500],
        **model_params
    }
    print()
    print(param_distributions)
    
    pipeline = Pipeline(steps=[
        ("vectorizer", vectorizer),
        ("pca", pca),
        ("normalize", scaler),
        ("model", model_)
    ])
    gs_model = RandomizedSearchCV(
        pipeline,
        param_distributions,
        scoring='f1',
        cv=n_splits_gs,
        refit=True,
        error_score='raise'
    )
    
    scores = cross_validate(
        gs_model,
        corpus,
        labels,
        cv=n_splits_cv,
        scoring=['accuracy', 'f1']
    )
    scores['model'] = [f"{model_name}"] * n_splits_cv
    if not(results):
        results = {key: [] for key in scores}
    for key in scores:
        results[key].extend(scores[key])

AttributeError: 'list' object has no attribute 'items'

## Comparação dos modelos
### Tabela

Em que os resultados são agrupados de acordo com os modelos e são calculadas média e desvio padrão dos dez modelos gerados para cada combinação vetorizador-método de aprendizagem de máquina.

In [ ]:
scores

In [ ]:
df_results = (
    pd
    .DataFrame(results)
    .groupby("model")
    .agg([np.mean, np.std])
    .transpose()
)

Nesta tabela temos informações relacionadas a:
- **Fit Time**: Tempo gasto na adequação do modelo ao conjunto de treino.
- **Score Time**: Tempo gasto para avaliação do modelo com o conjunto teste.
- **Test Accuracy**: Acurácia do modelo frente ao conjunto teste.
- **Test F1**: F1 score do modelo frente ao conjunto teste.

As linhas que computam a média dessas métricas estão coloridas de acordo com os valores, quão maiores, mais escuro o fundo da célula na tabela. 

In [ ]:
idx = pd.IndexSlice
slice_ = idx[idx[:,'mean'], :]
df_results.style.background_gradient(axis=1, subset=slice_)

### Figura

Para produzir a figura os dados provenientes dos resultados passaram por uma pequena modificação para que cada método de aprendizado e cada vetorizador ficasse em uma coluna específica.

In [ ]:
results_df = (
    pd
    .DataFrame(results)
)
results_df[['vectorizer', 'method']] = results_df['model'].str.split('-', expand=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,8))

sns.violinplot(x="method", y="test_accuracy", hue="vectorizer", 
               palette="colorblind", data=results_df, cut=0, ax=ax1)
ax1.set_title("Acurácia")
ax1.set_xlabel("Modelo de Predição")
ax1.set_ylabel("Acurácia dos conjuntos de teste")
ax1.legend(title="Vetorizador")

sns.violinplot(x="method", y="test_f1", hue="vectorizer", 
               palette="colorblind", data=results_df, cut=0, ax=ax2)
ax2.set_title("Comparação F1-Score")
ax2.set_xlabel("Modelo de Predição")
ax2.set_ylabel("F1-Score dos conjuntos de teste")
ax2.legend(title="Vetorizador")

plt.show()

Os gráficos do tipo violino exibem a distribuição métricas de avaliação explicadas em detalhe mais acima. Conforme também visto na tabela,a acurácia da maioria dos modelos se encontra acima de 90%. Isso vale para todos os modelos da validação cruzada feito para a Regressão Logística, o Classificador com Vetores de Suporte e a Random Forest. Esses métodos também mostraram bom desempenho em relação ao F1-Score, com valores acima de 0.91. 

A exceção de desempenho é vista para os modelos gerados com o método dos K Vizinhos Mais Próximos, o KNN. Esse método mostrou baixa acurácia, de cerca de 0.5 utilizando ambos os vetorizadores. Já o F1-Score, apesar de baixo para ambos, foi discrepante a depender do vetorizador. O modelo KNN que utilizou o vetorizador de contagem simples (bag of words, saco de palavras) teve F1-Score próximo a zero. Já o modelo que utilizou o vetorizador TFiDF, que considera também a quantidade de textos em que uma palavra aparece,atingiu um F1-Score de cerca de 0.67.